## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_weather_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_weather_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Nipawin,53.3668,-104.0009,23.04,58,100,11.50,CA,2022-03-27 21:44:57,overcast clouds
1,1,Mahebourg,-20.4081,57.7000,77.29,88,40,3.44,MU,2022-03-27 21:45:15,scattered clouds
2,2,Atuona,-9.8000,-139.0333,79.07,77,52,14.43,PF,2022-03-27 21:45:15,broken clouds
3,3,Punta Arenas,-53.1500,-70.9167,50.11,57,0,24.16,CL,2022-03-27 21:45:16,clear sky
4,4,Ushuaia,-54.8000,-68.3000,51.46,54,20,6.91,AR,2022-03-27 21:45:17,few clouds


In [46]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("How cold is too cold? "))
max_temp = float(input("Some like it hot, how hot do you like it? "))

How cold is too cold? 65
Some like it hot, how hot do you like it? 80


In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_cities_df = city_weather_data_df.loc[(city_weather_data_df["Max Temp"] <= max_temp) & \
                                       (city_weather_data_df["Max Temp"] >= min_temp)]
ideal_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
1,1,Mahebourg,-20.4081,57.7000,77.29,88,40,3.44,MU,2022-03-27 21:45:15,scattered clouds
2,2,Atuona,-9.8000,-139.0333,79.07,77,52,14.43,PF,2022-03-27 21:45:15,broken clouds
6,6,Maceio,-9.6658,-35.7353,78.24,88,75,4.61,BR,2022-03-27 21:45:18,broken clouds
7,7,Port Alfred,-33.5906,26.8910,67.17,89,99,1.99,ZA,2022-03-27 21:45:19,overcast clouds
12,12,Caravelas,-17.7125,-39.2481,76.55,82,0,5.41,BR,2022-03-27 21:45:22,clear sky
17,17,Port Elizabeth,-33.9180,25.5701,66.49,96,0,8.05,ZA,2022-03-27 21:45:25,clear sky
20,20,Kapaa,22.0752,-159.3190,78.78,65,0,6.91,US,2022-03-27 21:45:27,clear sky
26,26,Rikitea,-23.1203,-134.9692,79.11,68,5,10.36,PF,2022-03-27 21:45:31,clear sky
27,27,Hermanus,-34.4187,19.2345,67.80,68,97,3.56,ZA,2022-03-27 21:45:32,overcast clouds
29,29,Amapa,1.0000,-52.0000,73.42,99,100,2.53,BR,2022-03-27 21:45:34,heavy intensity rain


In [48]:
# 4a. Determine if there are any empty rows.
ideal_cities_df.count()

Unnamed: 0     243
City           243
Lat            243
Lng            243
Max Temp       243
Humidity       243
Cloudiness     243
Wind Speed     243
Country        238
Date           243
Description    243
dtype: int64

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = ideal_cities_df.dropna()

In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,77.29,scattered clouds,-20.4081,57.7000,
2,Atuona,PF,79.07,broken clouds,-9.8000,-139.0333,
6,Maceio,BR,78.24,broken clouds,-9.6658,-35.7353,
7,Port Alfred,ZA,67.17,overcast clouds,-33.5906,26.8910,
12,Caravelas,BR,76.55,clear sky,-17.7125,-39.2481,
17,Port Elizabeth,ZA,66.49,clear sky,-33.9180,25.5701,
20,Kapaa,US,78.78,clear sky,22.0752,-159.3190,
26,Rikitea,PF,79.11,clear sky,-23.1203,-134.9692,
27,Hermanus,ZA,67.80,overcast clouds,-34.4187,19.2345,
29,Amapa,BR,73.42,heavy intensity rain,1.0000,-52.0000,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [52]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,77.29,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
2,Atuona,PF,79.07,broken clouds,-9.8000,-139.0333,Villa Enata
6,Maceio,BR,78.24,broken clouds,-9.6658,-35.7353,Hotel Ponta Verde Maceió
7,Port Alfred,ZA,67.17,overcast clouds,-33.5906,26.8910,The Halyards Hotel
12,Caravelas,BR,76.55,clear sky,-17.7125,-39.2481,Pousada dos Navegantes
...,...,...,...,...,...,...,...
673,Adrar,MR,79.27,clear sky,20.5022,-10.0711,
674,Magadi,IN,74.48,clear sky,12.9667,77.2333,Gobi centre
680,Saint-Pierre,RE,76.91,few clouds,-21.3393,55.4781,Lindsey Hôtel
681,Weligama,LK,76.68,few clouds,5.9667,80.4167,Weligama Bay Resort


In [53]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


In [54]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                               dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))